# Time Series Forecasting Demo Setup

This document outlines the initial setup required to run the [Time Series Forecasting Demo on Snowflake](https://quickstarts.snowflake.com/guide/building_scalable_time_series_forecasting_models_on_snowflake/index.html#0).

## Overview

The purpose of this demo is to build scalable time series forecasting models directly within Snowflake using Python and Snowpark. This setup notebook:

- loads sample time series data
- Creates the necessary database objects (e.g., database, schema, stage)
- Loads the zip file that is required for the demo

## Next Steps

Once the setup notebook is complete, you can proceed to the model training and forecasting notebook of the [Repo](https://github.com/Snowflake-Labs/emerging-solutions-toolbox/blob/main/framework-forecast-model-builder/FORECAST_MODEL_BUILDER_DEPLOYMENT.ipynb?_fsi=jaVNHjij&_fsi=jaVNHjij&_fsi=jaVNHjij&_fsi=jaVNHjij).

---



### Load data

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
ts_data = pd.read_csv('data/tasty_bytes_ts_data.csv')
ts_data['DATE'] = pd.to_datetime(ts_data['DATE'])
ts_data.head()

In [ ]:
ts_sf = session.create_dataframe(ts_data)
ts_sf.write.mode("overwrite").save_as_table("tasty_bytes_sales")

In [ ]:
select * from tasty_bytes_sales
limit 10;

###  Create the stage for the zip file

This step can be done in the UI but this is for a faster approach

In [ ]:
-- If needed create a stage
-- If you already have a stage use that one

CREATE STAGE if not exists SALES_DATA 
	DIRECTORY = ( ENABLE = true ) 
	ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

### Load zip file to the stage

In [ ]:
MY_STAGE = 'SALES_DATA'
MY_FILE_NAME = "data/emerging-solutions-toolbox-main.zip"


# Upload the file to a stage.
put_result = session.file.put(MY_FILE_NAME, MY_STAGE, auto_compress=False,overwrite=True)
put_result[0].status